<a href="https://colab.research.google.com/github/jollygoodjacob/STF/blob/main/STF_starfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STARFM for Spatiotemporal Fusion

This Google Colab script is an implementation of the STARFM spatiotemporal fusion method, modified for use with UAV and Planet data.

## Install required packages

First, we need to install rasterio using pip, as Google Colab does not come preinstalled with this package.

In [1]:
!pip install rasterio

## Mount Google Drive

Next, we want to mount our Google Drive so that we can share data with our Google Colab script. We need both the starfm4py.py and parameters.py functions that are required to run the script, as well as our UAV and Planet data.

In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount= True)

Mounted at /content/drive


In [14]:
!cat /content/drive/MyDrive/STF/parameters.py

import numpy as np



# Set the size of the moving window in which the search for similar pixels 
# is performed
windowSize = 31

# Set the path where the results should be stored
path = 'STARFM_demo/'

# Set to True if you want to decrease the sensitivity to the spectral distance
logWeight = False

# If more than one training pairs are used, set to True
temp = False

# The spatial impact factor is a constant defining the relative importance of 
# spatial distance (in meters)
# Take a smaller value of the spatial impact factor for heterogeneous regions 
# (e.g. A = 150 m)
spatImp = 150 

# increasing the number of classes limits the number of similar pixels
numberClass = 4 

# Set the uncertainty value for the fine resolution sensor
# https://earth.esa.int/web/sentinel/technical-guides/sentinel-2-msi/performance 
uncertaintyFineRes = 0.03

# Set the uncertainty value for the coarse resolution sensor
# https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-3-olci/validat

## Load Functions for STARFM

In [15]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Add your module directory to the Python path
import sys
sys.path.append('/content/drive/MyDrive/STF')

# (Optional) Set working directory for reading TIFFs or output
import os
os.chdir('/content/drive/MyDrive/STF')

# Enable auto-reload of modules
%load_ext autoreload
%autoreload 2

# Try importing your values
from parameters import path, sizeSlices
print("Imported values:", path, sizeSlices)

# Try importing the STARFM module
import starfm4py as stp
print("Successfully imported starfm4py")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ModuleNotFoundError: No module named 'parameters'

In [12]:
import os
print(os.listdir('/content/drive/MyDrive/STF'))

['20220802_RGB_UAV.tif', '20220812_RGB_Planet.tif', '20220802_RGB_Planet.tif', 'starfm4py.py', 'parameters.py']


## Applt STARFM to our data

In [ ]:
# Import required packages

import time
import rasterio
import numpy as np
#import starfm4py as stp
import matplotlib.pyplot as plt
#from parameters import (path, sizeSlices)



start = time.time()

#Set the path where the images are stored
product = rasterio.open('/content/drive/MyDrive/STF/sim_STARFM_image.tif')
profile = product.profile
UAVt0 = rasterio.open('/content/drive/MyDrive/STF/20220802_UAV_RGB.tif').read(1)
Planett0 = rasterio.open('/content/drive/MyDrive/STF/20220802_182705_60_247e_3B_AnalyticMS_SR_8b_harmonized_clip.tif').read(1)
Planett1 = rasterio.open('/content/drive/MyDrive/STF/20220812_182402_83_2480_3B_AnalyticMS_SR_8b_harmonized_clip.tif').read(1)

# Set the path where to store the temporary results
path_fineRes_t0 = 'Temporary/Tiles_fineRes_t0/'
path_coarseRes_t0 = 'Temporary/Tiles_coarseRes_t0/'
path_coarseRes_t1 = 'Temporary/Tiles_fcoarseRes_t1/'

# Flatten and store the moving window patches
fine_image_t0_par = stp.partition(UAVT0, path_fineRes_t0)
coarse_image_t0_par = stp.partition(Planett0, path_coarseRes_t0)
coarse_image_t1_par = stp.partition(Planett1, path_coarseRes_t1)

print ("Done partitioning!")

# Stack the the moving window patches as dask arrays
S2_t0 = stp.da_stack(path_fineRes_t0, UAVT0.shape)
S3_t0 = stp.da_stack(path_coarseRes_t0, Planett0.shape)
S3_t1 = stp.da_stack(path_coarseRes_t1, Planett1.shape)

shape = (sizeSlices, UAVT0.shape[1])

print ("Done stacking!")

# Perform the prediction with STARFM
for i in range(0, UAVT0.size-sizeSlices*shape[1]+1, sizeSlices*shape[1]):

    fine_image_t0 = S2_t0[i:i+sizeSlices*shape[1],]
    coarse_image_t0 = S3_t0[i:i+sizeSlices*shape[1],]
    coarse_image_t1 = S3_t1[i:i+sizeSlices*shape[1],]
    prediction = stp.starfm(fine_image_t0, coarse_image_t0, coarse_image_t1, profile, shape)

    if i == 0:
        predictions = prediction

    else:
        predictions = np.append(predictions, prediction, axis=0)


# Write the results to a .tif file
print ('Writing product...')
profile = product.profile
profile.update(dtype='float64', count=1) # number of bands
file_name = path + 'prediction.tif'

result = rasterio.open(file_name, 'w', **profile)
result.write(predictions, 1)
result.close()


end = time.time()
print ("Done in", (end - start)/60.0, "minutes!")

# Display input and output
plt.imshow(UAVT0)
plt.gray()
plt.show()
plt.imshow(Planett0)
plt.gray()
plt.show()
plt.imshow(Planett1)
plt.gray()
plt.show()
plt.imshow(predictions)
plt.gray()
plt.show()